In [1]:
import sys, os, itertools
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
from gensim.corpora import Dictionary
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper
from models.lftm import LFTMwrapper
from models.bertopic2 import BERTopicWrapper
from sklearn.feature_extraction.text import CountVectorizer
from scipy.cluster import hierarchy as sch
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = preprocess_for_bow('data.csv', preprocessing=False)
bertopic=BERTopicWrapper(data['text'], data['ids'], custom_vectorizer=True)
#bertopic.model.fit(data['text'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [19]:
bertopic.find_topics('Virus origin')

([423, 654, 597, 653, 327],
 [0.5838068, 0.56007075, 0.55641145, 0.5460573, 0.52219105])

In [ ]:
# compute topic embedding
totalweight=0
embedding=0
for i in bertopic.find_topics('Virus origin')[0]:
    idxs = [j for j in range(len(bertopic.model.topics_)) if bertopic.model.topics_[j]==i]
    totalweight += i
    embedding += i*bertopic.umap_embeddings[idxs+1].sum(axis=0)/len(idxs) # assumes always label -1
embedding = embedding/totalweight


In [14]:
normalized_distance_matrix = 1 - (bertopic.dot_product_scores / (np.linalg.norm(bertopic.umap_embeddings, axis=1)[:, np.newaxis] * np.linalg.norm(bertopic.umap_embeddings, axis=1)))

In [34]:
bertopic.umap_embeddings[[2,4,7]].sum(axis=0)

array([25.499783 , 26.589165 , 38.183563 ,  4.159609 , -1.1610727],
      dtype=float32)